# RNN generator inference, GPU or CPU

In [0]:
from pathlib import Path
dataset_path = Path.home() / 'data/rnn_generator'; dataset_path
tmp_path = dataset_path / 'tmp/'

In [2]:
import torch

USE_GPU = torch.cuda.is_available(); 
# USE_GPU = False; 

print(f'USE_GPU={USE_GPU}')

def to_gpu(x, *args, **kwargs):
    return x.cuda(*args, **kwargs) if USE_GPU else x

USE_GPU=True


In [0]:
ALLCHARS, MODEL = ['all_characters', 'model']

fn_witkacy = {ALLCHARS: 'all_characters.p', MODEL: 'model.h100.l1.e2000.cpu.dat'}
# fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e2000.cpu.torch'}
# fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e101.gpu.torch'}
fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h400.l2.3000.gpu.torch'}


fn_dict = fn_pan_tadeusz

## Load all_characters list

In [4]:
import pickle

all_characters = pickle.load( open( tmp_path / fn_dict[ALLCHARS], 'rb' ) ); print(all_characters[:10])
n_characters = len(all_characters); print(n_characters)

['\n', ' ', '!', '"', '%', "'", '(', ')', ',', '-']
89


## Code

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(to_gpu(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [6]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(to_gpu(tensor))

print(char_tensor('ala ma kota'))

Variable containing:
 41
 52
 41
  1
 53
 41
  1
 51
 55
 59
 41
[torch.cuda.LongTensor of size 11 (GPU 0)]



In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [0]:
n_epochs = 300 # 3000
print_every = 100
plot_every = 1
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
print(decoder, flush=True)

## Load model

In [8]:
model_path_cpu = tmp_path / fn_dict[MODEL]
decoder = torch.load(model_path_cpu)
print(decoder, flush=True)

RNN(
  (encoder): Embedding(89, 400)
  (gru): GRU(400, 400, num_layers=2)
  (decoder): Linear(in_features=400, out_features=89)
)


In [0]:
decoder.gru.flatten_parameters()

## Generate text

In [0]:
# priming strings
# pan_tadeusz: kon, bę, Tad, Tadeusz

In [22]:
print(evaluate('Tadeusz', 1000, temperature=0.8))

Tadeusz ruce bysię zięcze wyką potaniu.

                      Niem połowie,
W korzał sługi trzele cocałem połują,
Podłużny, promiéj w krzykniut tak pióre,
Kostarskiém okonatała włachał który broda.


Bóż tę go pola parygie  rozmadali w czy Kostrzyła nie trono,
Stkowały myśliwycz w górą w kwitro ow rozyłu wrzałby i tylkaczy mował.

Wyba pocią piękiem utę piękrowaz nach natwotą wszystkiém, parte kielarz w goście
Na na to wławę w romadził karoki,
A taki w gości, i tylko z domu niech tali
Perna dą ba koło mu Aść kiedą kiedyń kut w remstaz tu tu,
I cię mił i rzezka kotyk,
A zoczasiem, patrze, rączą ktowytem takoły,
Aby drugi -- Sugrzy rował Wogruzechie,
Gt dardowszy oba zieśmie już zachłogą,
Parkusy z rogardość latem wyrzymowałby puszczeniéj Kodmercyfu utrzał
Że kamieni, popowiatrowie

Zowałamiał to gurł wszedwiadał go cię kwezowny,
Szął włos zdronie,
Tadeusz w zatrępiona,
Dostał Pan Sędzie Kuścich którdego w patrzał w stołowa zawrócił rupatem.

 ktroci, ztéj głowby,
Szczadać na który i go

In [21]:
print(evaluate('Tad', 1000, temperature=0.5))

Tadaladzie wyrzadzał wyrowie ziemię w toł kołowy
Stolach w okomna, trobaskiém Kozrokiem.

        cztart w ciągnął, krosi takiem,
Podania takie tarze w kapiastwi wrzał gościał
Przecież kto więto w tali wielkiéj koły w wieść u ogromował,
Jak tylko kołową wię karku Domeykie pańskił ziemiał oprawił
I w krwawy w tylecie i za wytarzą krowa,
Tada w koło, tak zatrał z nim zagrodzie.

                                                                                                                                                                                                                                                 Pan tém wyrzecie powierzeciał i wielki wyrzekł w tak,
A drzwi wyrzawie poworze powiatem,
A do tylko z Aśliwszy urzą krzykniemi.



Tam pa przysię w kołowy,
Dowada i się wię cię w tył w koleniu,
Przez gadał puszcie paroskim, rodku potowy
Szaka Litwę w nim do Litawie wyrzekł kuturom,
Który kołował w komny niedźwieciego trwał w goście rozmał gości.



Stanął Kuż krozał do włarny 

In [18]:
print(evaluate('bę', 200, temperature=1.4))

bę Aśoramisko;a
Po wiórnieeru somanaą niezas Dzykncowywyz, nióbł bręku jéj, zoczysłać, lepiwił Suknę lilicsty,
Parnałem Kasrzegu.
PaDezanąszdał Wę, Nigadrą nagu.
Gymiał Gdziaste, -- N czystawiny,
k? Tie
